In [1]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import datetime

import numpy as np




In [7]:
# !pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.0.1-cp37-none-any.whl size=10032 sha256=444d9a35b93b71a507c7b5b38d9e91bbe29f2c018b903b7e2ce4e7994653e635
  Stored in directory: /Users/ieunpyo/Library/Caches/pip/wheels/1d/0d/3b/6b9d4477a34b3905f246ff4e7acf6aafd4cc9b77d473629b77
Successfully built bayesian-optimization


# HYPERPARAMETER TUNING

## initial search

#### 고려해볼만한 변수는 다음과 같다.

#### n_estimators : Number of trees in random forest
#### max_features : The number of features to consider when looking for the best split
#### max_depth : Maximum number of levels in tree
#### min_samples_split : Minimum number of samples required to split a node
#### min_samples_leaf : Minimum number of samples required at each leaf node

#### Greed Search 에 필요한 시간이 충분치 않아서 Colab에서 팀원들과 hyperparameter space를 나누어 진행하였다.

n_estimators 분배 은표 [10,50] / 창현 [14,45] / 지현 [18,27,41] / 성호 [23,32,36]

In [55]:

'''
from google.colab import drive
drive.mount('/content/drive')

work_dir = '/content/drive/My Drive/work/gimhae_fire/'


# [10, 14, 18, 23, 27, 32, 36, 41, 45, 50]
# n_estimators 은표 [10,50] / 창현 [14,45] / 지현 [18,27,41] / 성호 [23,32,36]
n_estimators_list = [10,50]  # 자기가 돌리기로 한 부분을 리스트 형식으로 넣어주세요

# n =5 회 반복으로 하겠습니다.
n=5


Train_X = np.load(work_dir + 'Train_X.npy',allow_pickle=True)
Test_X = np.load(work_dir + 'Test_X.npy',allow_pickle=True)
Validation_X = np.load(work_dir + 'Validation_X.npy',allow_pickle=True)

Train_y = np.load(work_dir  + 'Train_y.npy',allow_pickle=True)
Validation_y = np.load(work_dir + 'Validation_y.npy',allow_pickle=True)

Train_X.shape


# n_estimators : Number of trees in random forest

# n_estimators = [int(x) for x in np.linspace(start = 10, stop = 50, num = 10)]

# max_features : The number of features to consider when looking for the best split
max_features = [int(x) for x in np.linspace(start = 10, stop = 30, num = 5)]

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10,100, num = 10)]
max_depth.append(None)

# min_samples_leaf : Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 5, 10]


# Create the random grid
random_grid = {'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_leaf': min_samples_leaf,
                }


random_grid


from sklearn.model_selection import ParameterGrid

def hyper_param_f1_score(n,n_estimators, max_features, max_depth, min_samples_leaf) :
    rfc = RandomForestClassifier(n_estimators=n_estimators, max_features=max_features, 
                                 max_depth=max_depth, min_samples_leaf=min_samples_leaf,
                                 n_jobs=-1)
    f1_score_list=[]
    for i in range(n) :
        rfc.fit(Train_X,Train_y)
        f1_score_list.append(f1_score(rfc.predict(Validation_X),Validation_y))
    m = np.mean(f1_score_list)
    s = np.std(f1_score_list)
    return [n_estimators, max_features, max_depth, min_samples_leaf, n, m,s]
    
    
result= []
for n_estimators_temp in n_estimators_list : 
    for i in range( len(list(ParameterGrid(random_grid)) ) ) :
        min_samples_leaf_temp = ParameterGrid(random_grid)[i]['min_samples_leaf']
        max_features_temp = ParameterGrid(random_grid)[i]['max_features']
        max_depth_temp = ParameterGrid(random_grid)[i]['max_depth']
    
        result.append(hyper_param_f1_score(n,n_estimators_temp, max_features_temp, max_depth_temp, min_samples_leaf_temp))
    

np.save(work_dir+'result',result,allow_pickle=True)
'''

##### Greed Search 결과부터 불러오기

In [160]:
result_10_50 = np.load('/Users/ieunpyo/PycharmProjects/Kaggle/gimhae_fire/result_10_50.npy',allow_pickle=True)
result_14_45 = np.load('/Users/ieunpyo/PycharmProjects/Kaggle/gimhae_fire/result_14_45.npy',allow_pickle=True)
result_18_27_41 = np.load('/Users/ieunpyo/PycharmProjects/Kaggle/gimhae_fire/result_18_27_41.npy',allow_pickle=True)



In [161]:
result_10_14_18_27_41_45_50 = np.r_[result_10_50,result_14_45,result_18_27_41]

In [162]:
result_10_14_18_27_41_45_50_pd = pd.DataFrame(result_10_14_18_27_41_45_50,columns=['n_estimators', 'max_features', 'max_depth', 'min_samples_leaf', 'n', 'mean','std'])


In [164]:
result_10_14_18_27_41_45_50_pd.sort_values(by='mean',ascending=False).head(20)

,n_estimators,max_features,max_depth,min_samples_leaf,n,mean,std
1055,45,15,100,1,5,0.392934,0.0158199
1086,45,20,None,2,5,0.391899,0.012256
320,50,30,20,1,5,0.391536,0.00797163
995,45,30,70,1,5,0.39129,0.0172644
1462,27,20,40,4,5,0.39097,0.0192686
1766,41,25,50,2,5,0.389622,0.0109563
910,45,20,40,1,5,0.389405,0.0196752
1716,41,25,30,2,5,0.38921,0.00812536
891,45,25,30,2,5,0.388652,0.00823097
960,45,20,60,1,5,0.388086,0.0157549


## second search
##### hyperparameter & fitting


In [ ]:
save_dir = '/Users/ieunpyo/PycharmProjects/Kaggle/gimhae_fire/'

In [ ]:

f1_best = 0
i = 0 
for i in range(100) : 
    # randomly select hyperparameters within reasonable range
    
    #n_estimators
    n_estimators_temp = np.random.randint(40,55)
    #max_features
    max_features_temp = np.random.randint(15,35)
    #min_sample_leaf
    min_sample_leaf_temp = np.random.randint(1,5)
    #max_depth
    max_depth_temp = np.random.randint(20,110)
    
    # Model
    RF_clf = RandomForestClassifier(n_estimators =n_estimators_temp, 
                            max_features = max_features_temp,
                            min_samples_leaf = min_sample_leaf_temp,
                            max_depth = max_depth_temp)
    
    RF_clf.fit(Train_X,Train_y)
    
    f1 = f1_score( RF_clf.predict(Validation_X), Validation_y ) 
    
    if f1 > f1_best : 
        # 새로 찍은 RF_clf의 f1 score가 best f1 score보다 높으면
        
        # 챔피언 바꾸기
        RF_clf_best = RF_clf
        f1_best = f1
        
        # 제출형태의 csv 파일로 저장
        submission = pd.DataFrame(RF_clf_best.predict(Test_X),columns=['fr_yn'])
        submission['fr_yn'] = submission['fr_yn'].map({1:'Y',0:'N'})
        now = datetime.datetime.now()
        submission.to_csv(save_dir+'화재예측과제_Submission' + '_'+now.strftime('%H:%M:%S') +'.csv',index=False)
        
    i = i+1
    
    




In [ ]:
RF_clf_best

In [ ]:
f1_score( RF_clf_best.predict(Validation_X), Validation_y ) 

## predict on test_X & submission
##### 정해진 hyperparameter로 N번 fitting 반복 predict -> f-score 최고점된 model로 제출

In [ ]:
submission = pd.DataFrame(RF_clf_best.predict(Test_X),columns=['fr_yn'])
submission['fr_yn'] = submission['fr_yn'].map({1:'Y',0:'N'})

In [ ]:
now = datetime.datetime.now()
submission.to_csv(save_dir+'화재예측과제_Submission' + '_'+now.strftime('%H:%M:%S') +'.csv',index=False)